In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import joblib 

In [2]:
df = pd.read_excel('ready_to_classify.xlsx')
df.head()

,Unnamed: 0,Carimbo de data/hora,id_integer,role,years_exp,org_size,use_metrics_planning,use_metrics_review,use_metrics_weekly,use_metrics_daily,use_metrics_retro,metrics_category,agile_methods_filtered,matched_metrics,not_matched_metrics
0,0,17:34:49,1,Product manager,0 a 5,Microempresa,Métricas,0,Métricas,0,0,"Cronograma e progresso, Produto, Tecnologia, C...","['Scrum', 'Kanban']",['ideal team capacity'],[]
1,1,17:57:23,2,Desenvolvedor(a),0 a 5,Microempresa,Métricas,Métricas,0,0,0,"Cronograma e progresso, Produto, Cliente","['Scrum', 'Kanban']","['nps', 'burndown', 'outstanding bugs', 'throu...",[]
2,2,20:50:58,3,Product manager,0 a 5,Pequena empresa,Gerenciamento de riscos,Métricas,0,Gerenciamento de riscos,"Gerenciamento de riscos, Métricas","Cronograma e progresso, Pessoas, Produto",['Scrum'],"['burndown', 'velocity']","['okr', 'quantidade de solicitações de clientes']"
3,3,22:26:06,4,Team leader,6 a 9,Microempresa,"Gerenciamento de riscos, Métricas",Gerenciamento de riscos,Gerenciamento de riscos,Gerenciamento de riscos,Métricas,"Cronograma e progresso, Produto, Cliente","['Scrum', 'Kanban']",['throughput'],"['cac', 'roadmaps', 'conversão']"
4,4,19:07:11,5,Product manager,0 a 5,Pequena empresa,"Gerenciamento de riscos, Métricas",Métricas,0,0,Métricas,Cronograma e progresso,"['Scrum', 'Kanban']","['user story points', 'nps']",[]


In [3]:
# Simplificar o dataset removendo colunas desnecessárias
df_cleaned = df[['role', 'years_exp', 'org_size', 'use_metrics_planning',
                 'use_metrics_review', 'use_metrics_weekly', 'use_metrics_daily',
                 'use_metrics_retro', 'agile_methods_filtered', 'metrics_category']]


In [4]:
# Definir as categorias únicas para classificação multilabel
categories = ['Cronograma e progresso', 'Produto', 'Processo', 'Tecnologia', 'Pessoas', 'Cliente']

# Criar colunas binárias para cada categoria
for category in categories:
    df_cleaned[category] = df_cleaned['metrics_category'].apply(lambda x: 1 if category in x else 0)

/var/folders/5j/0g6pzkwj7_g1p9xwqx5jfgq80000gn/T/ipykernel_92902/2031624344.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[category] = df_cleaned['metrics_category'].apply(lambda x: 1 if category in x else 0)
/var/folders/5j/0g6pzkwj7_g1p9xwqx5jfgq80000gn/T/ipykernel_92902/2031624344.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[category] = df_cleaned['metrics_category'].apply(lambda x: 1 if category in x else 0)
/var/folders/5j/0g6pzkwj7_g1p9xwqx5jfgq80000gn/T/ipykernel_929

In [5]:
# Explodir o dataframe para criar uma linha para cada categoria
exploded_data = []

# Loop por cada linha do dataframe
for idx, row in df_cleaned.iterrows():
    # Obter as categorias associadas a essa instância
    categories_present = [category for category in categories if row[category] == 1]
    
    # Criar uma nova linha para cada categoria presente
    for category in categories_present:
        new_row = row.copy()
        # Definir todas as categorias como 0, exceto a atual
        for cat in categories:
            new_row[cat] = 1 if cat == category else 0
        exploded_data.append(new_row)

# Converter a lista de linhas em um novo dataframe
df_exploded = pd.DataFrame(exploded_data)

In [6]:
# One-hot encoding para variáveis categóricas
df_exploded_encoded = pd.get_dummies(df_exploded, columns=['role', 'years_exp', 'org_size'])


In [7]:
# Tratar as colunas relacionadas ao uso de métricas
metrics_columns = ['use_metrics_planning', 'use_metrics_review', 'use_metrics_weekly', 'use_metrics_daily', 'use_metrics_retro']

# Atualizar as colunas para considerar 1 sempre que houver algum valor presente, e 0 caso contrário
for col in metrics_columns:
    df_exploded_encoded[col] = df_exploded[col].apply(lambda x: 1 if x != 0 and pd.notna(x) else 0)

In [8]:
import ast

# Verificar se o conteúdo da coluna 'agile_methods_filtered' está no formato correto
def method_present(method, method_list_str):
    # Verificar se a string está no formato de lista e converter para uma lista real
    try:
        method_list = ast.literal_eval(method_list_str)  # Converte a string da lista em uma lista Python
    except (ValueError, SyntaxError):
        method_list = []  # Se a conversão falhar, retornar uma lista vazia
    return 1 if method in method_list else 0

# Expandir a coluna 'agile_methods_filtered' para colunas binárias
agile_methods = ['Scrum', 'Kanban', 'Lean', 'XP', 'Safe', 'ScrumBan']

# Para cada método ágil, criar uma nova coluna binária
for method in agile_methods:
    df_exploded_encoded[method] = df_exploded['agile_methods_filtered'].apply(lambda x: method_present(method, x))

# Excluir a coluna 'agile_methods_filtered' após a conversão
df_exploded_encoded = df_exploded_encoded.drop(columns=['agile_methods_filtered'])


In [9]:
# Excluir a coluna 'metrics_category' para evitar o erro com string
df_exploded_encoded = df_exploded_encoded.drop(columns=['metrics_category'])

In [10]:
df_exploded_encoded.to_excel('df_exploded_encoded.xlsx')

In [11]:
# Salvar a estrutura das colunas do treino para usar no futuro
column_structure = list(df_exploded_encoded.columns)

# Salvar as colunas usadas durante o treinamento para referência futura
joblib.dump(column_structure, 'column_structure.joblib')

['column_structure.joblib']

In [12]:
X = df_exploded_encoded.drop(columns=categories)  # Features
classification_reports = {}  # Dicionário para armazenar relatório

In [13]:
from sklearn.metrics import classification_report

In [14]:
from imblearn.over_sampling import SMOTE


In [15]:
# Loop para cada categoria de métricas
for category in categories:
    # Definir as features e o alvo
    X = df_exploded_encoded.drop(columns=categories)
    y = df_exploded_encoded[category]

    # Dividir os dados em treino+validação (80%) e teste (20%)
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Dividir o treino+validação em treino (60%) e validação (20%)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

    # Rebalancear o dataset usando SMOTE para o conjunto de treino
    sm = SMOTE(random_state=42)
    X_res, y_res = sm.fit_resample(X_train, y_train)

    # Definir hiperparâmetros para ajuste com GridSearchCV
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2']
    }

    # Usar F1-score como métrica de avaliação para maximizar a previsão positiva
    grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42, class_weight='balanced'), 
                               param_grid=param_grid, 
                               cv=3, 
                               scoring='f1',  # Maximizar o F1-score para melhor equilíbrio entre precision e recall
                               n_jobs=-1, 
                               verbose=2)

    # Ajustar o modelo com os dados reamostrados
    grid_search.fit(X_res, y_res)

    # Melhor modelo após ajuste de hiperparâmetros
    best_rf = grid_search.best_estimator_

    # Avaliar no conjunto de validação
    probabilities = best_rf.predict_proba(X_val)[:, 1]
    threshold = 0.3  # Ajuste do threshold para aumentar as previsões positivas
    predicted_classes = (probabilities >= threshold).astype(int)

    # Exibir as métricas de avaliação no conjunto de validação
    print(f"Resultados para a categoria: {category}")
    print(classification_report(y_val, predicted_classes))

    # Salvar o melhor modelo treinado
    model_filename = f"modelo_{category}_ajustado.joblib"
    joblib.dump(best_rf, model_filename)
    print(f"Modelo para {category} salvo como {model_filename}")

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.8s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.8s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.8s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   3.2s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   3.4s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   3.3s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   1.7s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=1

/Users/viniciuslunardifarias/projects/ufsc/tcc/tcc_env/lib/python3.12/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Resultados para a categoria: Cronograma e progresso
              precision    recall  f1-score   support

           0       0.62      0.32      0.42       103
           1       0.16      0.39      0.22        33

    accuracy                           0.34       136
   macro avg       0.39      0.36      0.32       136
weighted avg       0.51      0.34      0.37       136

Modelo para Cronograma e progresso salvo como modelo_Cronograma e progresso_ajustado.joblib
Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.6s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.6s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.6s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total

KeyboardInterrupt: 